In [1]:
import xarray as xr
import numpy as np
import cftime
import nc_time_axis
import pandas as pd
import matplotlib.pyplot as plt 
from matplotlib import ticker, cm
#from cftime import datetime 
from datetime import datetime, timedelta
#import datetime as dt
import matplotlib.colors as colors
import math
import random
import matplotlib.cm as mcm
from matplotlib.ticker import MaxNLocator
#jet = mcm.get_cmap('jet')
jet = mcm.get_cmap('jet') if isinstance(mcm.get_cmap('jet'), str) else mcm.get_cmap('jet')
import netCDF4 as nc
import sys
import os
import psutil
import netCDF4 as nc
import cartopy.crs as ccrs # CRS stands for "Coordinate reference systems" for map projection
from cartopy.crs import PlateCarree
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from dateutil import tz
import pytz
import time
from time import process_time
from tqdm import tqdm
from dask.distributed import LocalCluster, Client
import dask.array as da
import dask.dataframe as dd
import dask
from dask import delayed
from dask.diagnostics import ProgressBar  # Import the ProgressBar class
import calendar
%matplotlib inline 
#import line_profiler
#%load_ext line_profiler

/tmp/ipykernel_2646693/2706121075.py:17: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  jet = mcm.get_cmap('jet') if isinstance(mcm.get_cmap('jet'), str) else mcm.get_cmap('jet')


In [2]:
################################################################################
# Open WACCM dataset to define bins
################################################################################


# Define the relative path to the symbolic link
symbolic_link = 'Nc_Files/ACP_CESM213_FX2000_f19_f19_mg16_Na_Fe_Mg_iontransport'

# Get the full path to the symbolic link
link_path = os.path.expanduser(os.path.expandvars(os.path.normpath(symbolic_link)))

# Join the symbolic link with the specific file you want to open
file_name = 'ACP_CESM213_FX2000_f19_f19_mg16_Na_Fe_Mg_iontransport.cam.h2.0001-01-01-00000.nc'
file_path = os.path.join(link_path, file_name)

# Open the file
wds = xr.open_dataset(file_path)
#wds

In [3]:
wds_lat = wds['lat']
wds_lon = wds['lon']
wds_lev = wds['lev']
wds_ilev = wds['ilev']
wds_geopH = wds.variables['Z3'] / 1000 #m-> km
wds_Zavg = wds_geopH.mean(('time','lat', 'lon'))
wds_time = wds['time']

# Slice arrays (lev, altitude) between chosen range (~80-130km)
lev_sl_idx_min = 42
lev_sl_idx_max = 66

# Slice arrays (lev & alt) between chosen range & print
lev_sl = wds_lev[lev_sl_idx_min:lev_sl_idx_max+1]
Zavg_sl = wds_Zavg[lev_sl_idx_min:lev_sl_idx_max+1]


print('    Array lev = ' + str("%.1e" % wds_lev[lev_sl_idx_max] ) + 'hPa : ' + str("%.1e" % wds_lev[lev_sl_idx_min] ) + 'hPa'
           + ' (approx ' + str("%.0f" % wds_Zavg[lev_sl_idx_max]) + 'km : ' + str("%.0f" % wds_Zavg[lev_sl_idx_min] ) + 'km' + ')'   )


    Array lev = 6.0e-03hPa : 1.5e-05hPa (approx 81km : 126km)


In [4]:
################################################################################
# Lat bins
################################################################################

lat_values = wds_lat.values

lat_bin_edges = lat_values
lat_bin_midpoints = (lat_bin_edges[:-1] + lat_bin_edges[1:]) / 2
#lat_bin_widths = np.diff(lat_bin_edges)
#lat_average_bin_width = np.mean(lat_bin_widths)
lat_num_bins = len(lat_bin_midpoints)


# # ##Print the results
# print("Lat Bin Edges (Lower and Upper):")
# for i in range(len(lat_bin_edges) - 1):
#     print(f"Bin {i + 1}: [{lat_bin_edges[i]}, {lat_bin_edges[i + 1]}]")

print("\nLat Bin Midpoints:")
for i in range(len(lat_bin_midpoints)):
    print(f"Bin {i + 1} Midpoint: {lat_bin_midpoints[i]}")

# print("\nLat Bin Widths:", lat_bin_widths)
# print(f"Lat Average Bin Width: {lat_average_bin_width:.6f} degrees")
# print(f"Lat Number of Bins: {lat_num_bins}")


# # Lat Average Bin Width: 1.894737 degrees
# # Lat Number of Bins: 95


Lat Bin Midpoints:
Bin 1 Midpoint: -89.05263157894737
Bin 2 Midpoint: -87.15789473684211
Bin 3 Midpoint: -85.26315789473685
Bin 4 Midpoint: -83.36842105263158
Bin 5 Midpoint: -81.47368421052632
Bin 6 Midpoint: -79.57894736842105
Bin 7 Midpoint: -77.6842105263158
Bin 8 Midpoint: -75.78947368421052
Bin 9 Midpoint: -73.89473684210526
Bin 10 Midpoint: -72.0
Bin 11 Midpoint: -70.10526315789474
Bin 12 Midpoint: -68.21052631578948
Bin 13 Midpoint: -66.31578947368422
Bin 14 Midpoint: -64.42105263157895
Bin 15 Midpoint: -62.526315789473685
Bin 16 Midpoint: -60.631578947368425
Bin 17 Midpoint: -58.736842105263165
Bin 18 Midpoint: -56.8421052631579
Bin 19 Midpoint: -54.94736842105263
Bin 20 Midpoint: -53.05263157894737
Bin 21 Midpoint: -51.15789473684211
Bin 22 Midpoint: -49.26315789473684
Bin 23 Midpoint: -47.36842105263158
Bin 24 Midpoint: -45.473684210526315
Bin 25 Midpoint: -43.578947368421055
Bin 26 Midpoint: -41.684210526315795
Bin 27 Midpoint: -39.78947368421053
Bin 28 Midpoint: -37.89473

In [5]:
# ################################################################################
# # Lon bins 0 -> 360
# ################################################################################

lon_upper_edge_last_bin = 360.0
lon_bin_edges = np.append(wds_lon, lon_upper_edge_last_bin)

lon_bin_midpoints = (lon_bin_edges[:-1] + lon_bin_edges[1:]) / 2
lon_bin_widths = np.diff(lon_bin_edges)
lon_average_bin_width = np.mean(lon_bin_widths)
lon_num_bins = len(lon_bin_midpoints)


# ## Print the results
# print("Lon Bin Edges (Lower and Upper):")
# for i in range(len(lon_bin_edges) - 1):
#     print(f"Bin {i + 1}: [{lon_bin_edges[i]}, {lon_bin_edges[i + 1]}]")

print("\nLon Bin Midpoints:")
for i in range(len(lon_bin_midpoints)):
    print(f"Lon Bin {i + 1} Midpoint: {lon_bin_midpoints[i]}")

# print("\nLon Bin Widths:", lon_bin_widths)
# print(f"Average Lon Bin Width: {lon_average_bin_width:.8f} degrees")
# print(f"Number of Lon Bins: {lon_num_bins}")


# # Average Lon Bin Width: 2.50000000 degrees
# # Number of Lon Bins: 144


Lon Bin Midpoints:
Lon Bin 1 Midpoint: 1.25
Lon Bin 2 Midpoint: 3.75
Lon Bin 3 Midpoint: 6.25
Lon Bin 4 Midpoint: 8.75
Lon Bin 5 Midpoint: 11.25
Lon Bin 6 Midpoint: 13.75
Lon Bin 7 Midpoint: 16.25
Lon Bin 8 Midpoint: 18.75
Lon Bin 9 Midpoint: 21.25
Lon Bin 10 Midpoint: 23.75
Lon Bin 11 Midpoint: 26.25
Lon Bin 12 Midpoint: 28.75
Lon Bin 13 Midpoint: 31.25
Lon Bin 14 Midpoint: 33.75
Lon Bin 15 Midpoint: 36.25
Lon Bin 16 Midpoint: 38.75
Lon Bin 17 Midpoint: 41.25
Lon Bin 18 Midpoint: 43.75
Lon Bin 19 Midpoint: 46.25
Lon Bin 20 Midpoint: 48.75
Lon Bin 21 Midpoint: 51.25
Lon Bin 22 Midpoint: 53.75
Lon Bin 23 Midpoint: 56.25
Lon Bin 24 Midpoint: 58.75
Lon Bin 25 Midpoint: 61.25
Lon Bin 26 Midpoint: 63.75
Lon Bin 27 Midpoint: 66.25
Lon Bin 28 Midpoint: 68.75
Lon Bin 29 Midpoint: 71.25
Lon Bin 30 Midpoint: 73.75
Lon Bin 31 Midpoint: 76.25
Lon Bin 32 Midpoint: 78.75
Lon Bin 33 Midpoint: 81.25
Lon Bin 34 Midpoint: 83.75
Lon Bin 35 Midpoint: 86.25
Lon Bin 36 Midpoint: 88.75
Lon Bin 37 Midpoint: 

In [6]:
# ################################################################################
# # Lon bins -180 -> 180
# ################################################################################

# Updated value for a range of -180 to +180 degrees
lon_upper_edge_last_bin = 180.0
lon_bin_edges = np.append(np.arange(-180.0, 180.0, 2.5), lon_upper_edge_last_bin)
lon_bin_midpoints = (lon_bin_edges[:-1] + lon_bin_edges[1:]) / 2

# Other calculations (width, number of bins, etc.) remain the same
lon_bin_widths = np.diff(lon_bin_edges)
lon_average_bin_width = np.mean(lon_bin_widths)
lon_num_bins = len(lon_bin_midpoints)

# # Print the results
# print("Lon Bin Edges (Lower and Upper):")
# for i in range(len(lon_bin_edges) - 1):
#     print(f"Bin {i + 1}: [{lon_bin_edges[i]}, {lon_bin_edges[i + 1]}]")

print("\nLon Bin Midpoints:")
for i in range(len(lon_bin_midpoints)):
    print(f"Lon Bin {i + 1} Midpoint: {lon_bin_midpoints[i]}")

# print("\nLon Bin Widths:", lon_bin_widths)
# print(f"Average Lon Bin Width: {lon_average_bin_width:.8f} degrees")
# print(f"Number of Lon Bins: {lon_num_bins}")


Lon Bin Midpoints:
Lon Bin 1 Midpoint: -178.75
Lon Bin 2 Midpoint: -176.25
Lon Bin 3 Midpoint: -173.75
Lon Bin 4 Midpoint: -171.25
Lon Bin 5 Midpoint: -168.75
Lon Bin 6 Midpoint: -166.25
Lon Bin 7 Midpoint: -163.75
Lon Bin 8 Midpoint: -161.25
Lon Bin 9 Midpoint: -158.75
Lon Bin 10 Midpoint: -156.25
Lon Bin 11 Midpoint: -153.75
Lon Bin 12 Midpoint: -151.25
Lon Bin 13 Midpoint: -148.75
Lon Bin 14 Midpoint: -146.25
Lon Bin 15 Midpoint: -143.75
Lon Bin 16 Midpoint: -141.25
Lon Bin 17 Midpoint: -138.75
Lon Bin 18 Midpoint: -136.25
Lon Bin 19 Midpoint: -133.75
Lon Bin 20 Midpoint: -131.25
Lon Bin 21 Midpoint: -128.75
Lon Bin 22 Midpoint: -126.25
Lon Bin 23 Midpoint: -123.75
Lon Bin 24 Midpoint: -121.25
Lon Bin 25 Midpoint: -118.75
Lon Bin 26 Midpoint: -116.25
Lon Bin 27 Midpoint: -113.75
Lon Bin 28 Midpoint: -111.25
Lon Bin 29 Midpoint: -108.75
Lon Bin 30 Midpoint: -106.25
Lon Bin 31 Midpoint: -103.75
Lon Bin 32 Midpoint: -101.25
Lon Bin 33 Midpoint: -98.75
Lon Bin 34 Midpoint: -96.25
Lon B

In [7]:
################################################################################
# Zavg_sl bins - Note first bin is at top
################################################################################

# Convert xarray Variable to NumPy array
Zavg_sl_values = Zavg_sl.values

Zavg_sl_bin_widths = np.diff(Zavg_sl_values)
Zavg_sl_bin_edges = [Zavg_sl_values[0] - Zavg_sl_bin_widths[0] / 2] + [Zavg_sl_values[i] + Zavg_sl_bin_widths[i] / 2 for i in range(len(Zavg_sl_bin_widths))] + [Zavg_sl_values[-1] + Zavg_sl_bin_widths[-1] / 2]
Zavg_sl_bin_midpoints = [(Zavg_sl_bin_edges[i] + Zavg_sl_bin_edges[i + 1]) / 2 for i in range(len(Zavg_sl_bin_edges) - 1)]
#Zavg_sl_bin_widths = [Zavg_sl_bin_edges[i + 1] - Zavg_sl_bin_edges[i] for i in range(len(Zavg_sl_bin_edges) - 1)]
Zavg_sl_num_bins = len(Zavg_sl_bin_midpoints)


# # Print the bin edges and midpoints
# print("Zavg_sl Bin Edges (Lower and Upper):")
# for i in range(len(Zavg_sl_bin_edges) - 1):
#     print(f"Bin {i + 1}: [{Zavg_sl_bin_edges[i]}, {Zavg_sl_bin_edges[i + 1]}")

print("\nZavg_sl Bin Midpoints:")
for i in range(len(Zavg_sl_bin_midpoints)):
    print(f"Bin {i + 1} Midpoint: {Zavg_sl_bin_midpoints[i]}")

# # Print the bin widths
# print("\nZavg_sl Bin Widths:")
# for i in range(len(Zavg_sl_bin_widths)):
#     print(f"Bin {i + 1} Width: {Zavg_sl_bin_widths[i]}")

# print(f"Number of Zavg_sl_bin_widths Bins: {Zavg_sl_num_bins}")

#Number of Zavg_sl_bin_widths Bins: 25


Zavg_sl Bin Midpoints:
Bin 1 Midpoint: 126.2462158203125
Bin 2 Midpoint: 122.4520320892334
Bin 3 Midpoint: 118.93509101867676
Bin 4 Midpoint: 115.77444839477539
Bin 5 Midpoint: 112.94672012329102
Bin 6 Midpoint: 110.42250442504883
Bin 7 Midpoint: 108.16627883911133
Bin 8 Midpoint: 106.13789558410645
Bin 9 Midpoint: 104.29532623291016
Bin 10 Midpoint: 102.59773063659668
Bin 11 Midpoint: 101.00767707824707
Bin 12 Midpoint: 99.49256706237793
Bin 13 Midpoint: 98.02574348449707
Bin 14 Midpoint: 96.58683586120605
Bin 15 Midpoint: 95.16112518310547
Bin 16 Midpoint: 93.73861694335938
Bin 17 Midpoint: 92.31366157531738
Bin 18 Midpoint: 90.8852310180664
Bin 19 Midpoint: 89.45649719238281
Bin 20 Midpoint: 88.03251647949219
Bin 21 Midpoint: 86.61766815185547
Bin 22 Midpoint: 85.21463966369629
Bin 23 Midpoint: 83.82406616210938
Bin 24 Midpoint: 82.44376945495605
Bin 25 Midpoint: 81.06900787353516


In [8]:
LT_bin_edges = np.linspace(0, 24, 49)
LT_bin_edges[-1] = 24.0

# Calculate bin midpoints
LT_bin_midpoints = (LT_bin_edges[:-1] + LT_bin_edges[1:]) / 2

# Calculate the number of bins
LT_num_bins = len(LT_bin_edges) - 1

# Calculate bin widths
LT_bin_widths = np.diff(LT_bin_edges)

# Calculate the average bin width
average_bin_width = np.sum(LT_bin_widths) / LT_num_bins


# ##Print the results
# print("LT Bin Edges (Lower and Upper):")
# for i in range(len(LT_bin_edges) - 1):
#     print(f"Bin {i + 1}: [{LT_bin_edges[i]}, {LT_bin_edges[i + 1]}]")

print("\nLT Bin Midpoints:")
for i in range(len(LT_bin_midpoints)):
    print(f"Bin {i + 1} Midpoint: {LT_bin_midpoints[i]}")

# print("\nNumber of LT Bins:")
# print(LT_num_bins)

# # Print the average bin width
# print("Average LT Bin Width:", average_bin_width)


LT Bin Midpoints:
Bin 1 Midpoint: 0.25
Bin 2 Midpoint: 0.75
Bin 3 Midpoint: 1.25
Bin 4 Midpoint: 1.75
Bin 5 Midpoint: 2.25
Bin 6 Midpoint: 2.75
Bin 7 Midpoint: 3.25
Bin 8 Midpoint: 3.75
Bin 9 Midpoint: 4.25
Bin 10 Midpoint: 4.75
Bin 11 Midpoint: 5.25
Bin 12 Midpoint: 5.75
Bin 13 Midpoint: 6.25
Bin 14 Midpoint: 6.75
Bin 15 Midpoint: 7.25
Bin 16 Midpoint: 7.75
Bin 17 Midpoint: 8.25
Bin 18 Midpoint: 8.75
Bin 19 Midpoint: 9.25
Bin 20 Midpoint: 9.75
Bin 21 Midpoint: 10.25
Bin 22 Midpoint: 10.75
Bin 23 Midpoint: 11.25
Bin 24 Midpoint: 11.75
Bin 25 Midpoint: 12.25
Bin 26 Midpoint: 12.75
Bin 27 Midpoint: 13.25
Bin 28 Midpoint: 13.75
Bin 29 Midpoint: 14.25
Bin 30 Midpoint: 14.75
Bin 31 Midpoint: 15.25
Bin 32 Midpoint: 15.75
Bin 33 Midpoint: 16.25
Bin 34 Midpoint: 16.75
Bin 35 Midpoint: 17.25
Bin 36 Midpoint: 17.75
Bin 37 Midpoint: 18.25
Bin 38 Midpoint: 18.75
Bin 39 Midpoint: 19.25
Bin 40 Midpoint: 19.75
Bin 41 Midpoint: 20.25
Bin 42 Midpoint: 20.75
Bin 43 Midpoint: 21.25
Bin 44 Midpoint: 21.7

In [9]:
### Raw satellite data ###
filepath = 'Nc_Files/s4max_files/mod_s4max.nc'

### Processed time data into datetime format ###
#new_filepath = 'Nc_Files/s4max_files/mod_s4max.nc'

### My processed data - 80-130km only ###
#result_filepath = 'Nc_Files/s4max_files/mod_s4max_80-130km.nc'

ds = xr.open_dataset(filepath)
ds

<xarray.Dataset>
Dimensions:       (nfile: 9266536)
Coordinates:
    altitude      (nfile) float64 ...
    lat           (nfile) float64 ...
    lon           (nfile) float64 ...
    time          (nfile) datetime64[ns] ...
    season        (nfile) object ...
    month         (nfile) object ...
Dimensions without coordinates: nfile
Data variables: (12/14)
    Ut_time       (nfile) float64 ...
    DOY           (nfile) float64 ...
    LT            (nfile) float64 ...
    mlat          (nfile) float64 ...
    mlon          (nfile) float64 ...
    mlt           (nfile) float64 ...
    ...            ...
    month_var     (nfile) object ...
    season_var    (nfile) object ...
    lat_var       (nfile) float64 ...
    lon_var       (nfile) float64 ...
    altitude_var  (nfile) float64 ...
    time_var      (nfile) datetime64[ns] ...

In [23]:

###########################################################################################################################

s4max_threshold = 0.5

#Arras and Wickert 2018 s4max > 0.2
#Yue et al 2016 s4max > 0.3
#Yue et al 2015 s4max > 0.3
#Ray 2006 s4max > 0.4
#Ko and Yeh 2010 'strong scintillations' defined as smax >= 0.5

#=====================================================================
# Set the desired months
# desired_months = [3,4,5]  
# desired_months = [6,7,8]  
# desired_months = [9,10,11]
desired_months = [12,1,2] 

averaging_period = ""
# Calculate the desired averaging_period based on your logic
if len(desired_months) == 1:
    averaging_period = "Monthly"
elif len(desired_months) == 2:
    averaging_period = "Two-Month"
elif len(desired_months) == 3:
    averaging_period = "Three-Month"
else:
    averaging_period = "Unknown"  # You can specify another value for other cases

season = ""
if set(desired_months) <= {12, 1, 2}:
    season = "winter"
elif set(desired_months) <= {3, 4, 5}:
    season = "spring"
elif set(desired_months) <= {6, 7, 8}:
    season = "summer"
elif set(desired_months) <= {9, 10, 11}:
    season = "autumn"    
else:
    season = "unknown"
#===================================================================== 
freq_LT = np.zeros((Zavg_sl_num_bins, lat_num_bins, LT_num_bins))
nmeas_LT = np.zeros((Zavg_sl_num_bins, lat_num_bins, LT_num_bins))

freq_lon = np.zeros((Zavg_sl_num_bins, lat_num_bins, lon_num_bins))
nmeas_lon = np.zeros((Zavg_sl_num_bins, lat_num_bins, lon_num_bins))

nfile = len(ds['lat'])
ds_length = np.arange(nfile)
#====================================================================

for i in tqdm(ds_length):
    current_altitude = ds['altitude_var'].values[i]
    current_lat = ds['lat'].values[i]
    current_lon = ds['lon'].values[i]
    current_LT = ds["LT"].values[i]  
    current_s4max = ds['s4max'].values[i]
    
    current_time = ds["time"].values[i]
    current_month = int(np.datetime64(current_time, 'M').astype('M8[M]').view('i8')) % 12 + 1  # Extract month as an integer (1-12)
  
    if current_month in desired_months:
        if Zavg_sl_bin_edges[-1] <= current_altitude <= Zavg_sl_bin_edges[0]: # Check if the current_altitude is within the range of defined bins

            altitude_bin_index = np.digitize(current_altitude, Zavg_sl_bin_edges, right=False) - 1
            lat_bin_index = np.digitize(current_lat, lat_bin_edges) - 1
            lon_bin_index = np.digitize(current_lon, lon_bin_edges) - 1
            if current_lon == lon_bin_edges[-1]:
                lon_bin_index = len(lon_bin_edges) - 2
            LT_bin_index = np.digitize(current_LT, LT_bin_edges) - 1
            if current_LT == LT_bin_edges[-1]:
                LT_bin_index = len(LT_bin_edges) - 2
            
            #print('loop no = ' + str(i) )
            
            nmeas_lon[altitude_bin_index, lat_bin_index, lon_bin_index] = nmeas_lon[altitude_bin_index, lat_bin_index, lon_bin_index] + 1.
            nmeas_LT[altitude_bin_index, lat_bin_index, LT_bin_index] = nmeas_LT[altitude_bin_index, lat_bin_index, LT_bin_index] + 1.

            if current_s4max > s4max_threshold:
                freq_lon[altitude_bin_index, lat_bin_index, lon_bin_index] = freq_lon[altitude_bin_index, lat_bin_index, lon_bin_index] + 1.
                freq_LT[altitude_bin_index, lat_bin_index, LT_bin_index] = freq_LT[altitude_bin_index, lat_bin_index, LT_bin_index] + 1.
            
###########################################################################################################################    
# Calculate occurence frequency  
    
ocfr_lon = np.full((Zavg_sl_num_bins, lat_num_bins, lon_num_bins), 0.)
ocfr_LT = np.full((Zavg_sl_num_bins, lat_num_bins, LT_num_bins), 0.)

# Check if arrays are equal to 0
lon_mask = (freq_lon != 0.) & (nmeas_lon != 0.)
LT_mask = (freq_LT != 0.) & (nmeas_LT != 0.)

# Calculate ocfr_lon and ocfr_LT only where the mask is True
if lon_mask.any():
    ocfr_lon[lon_mask] = (freq_lon[lon_mask] / nmeas_lon[lon_mask]) * 100.0

if LT_mask.any():
    ocfr_LT[LT_mask] = (freq_LT[LT_mask] / nmeas_LT[LT_mask]) * 100.0


###########################################################################################################################
# Save occurence frequency to nc file

Ocfr_ds = xr.Dataset(
    data_vars={
        "ocfr_lon": (['altitude', 'latitude', 'longitude'], ocfr_lon),
        "ocfr_LT": (['altitude', 'latitude', 'local_time'], ocfr_LT)
    },
    coords={
        "altitude": Zavg_sl_bin_midpoints,
        "latitude": lat_bin_midpoints,
        "longitude": lon_bin_midpoints,
        "local_time": LT_bin_midpoints,
    },
    attrs={
        'averaging_period': averaging_period,
        'season': season
        
    }
                )

output_directory ="./Nc_Files/s4max/"

output_file = f"{output_directory}Ocfr_{season}_s4max_{s4max_threshold}.nc"
Ocfr_ds.to_netcdf(output_file)
                
###########################################################################################################################
    

100%|███████████████████████████████████████████████████████████████████████████████████████| 9266536/9266536 [11:52<00:00, 12998.53it/s]
